# Genomic Data Processing Pipeline
This notebook processes genomic VCF data using Hail for machine learning preparation.
## Setup and Requirements
- Hail
- Pandas
- Python 3.9+
## Pipeline Overview
1. Data Loading
2. Quality Control
3. Feature Extraction
## 1. Initial Setup
First, we'll import our required libraries and set up logging utilities.

## Initialize Hail for local processing

In [ ]:
from src.utils.hail_utils import initialize_hail
from src.preprocessing.vcf_loader import load_vcf
from src.utils.logging_utils import log_step

# Initialize Hail
initialize_hail()

# Load your VCF file
vcf_path = r'C:\Users\jmcdonald\Desktop\Projects\The Genome Project\gnomad.exomes.v4.1.sites.chr13.vcf.bgz'
mt = load_vcf(vcf_path)

# Continue with analysis...

Loading BokehJS ...

Initializing Hail with default parameters...
Running on Apache Spark version 3.5.4
SparkUI available at http://LT2058-20241028.SHTC.com:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.133-4c60fddb171a
LOGGING: writing to c:\Users\jmcdonald\source\PersonalRepos\genome-analysis-ml\notebooks\hail-20250114-1513-0.2.133-4c60fddb171a.log


Hail is already initialized


In [16]:
# # Configure your VCF file path
# vcf_path = r'C:\Users\jmcdonald\Desktop\Projects\The Genome Project\gnomad.exomes.v4.1.sites.chr13.vcf.bgz'
# print(f"Checking if file exists: {os.path.exists(vcf_path)}")

# # Check file existence and size
# print(f"Checking file details:")
# print(f"File exists: {os.path.exists(vcf_path)}")
# if os.path.exists(vcf_path):
#     print(f"File size: {os.path.getsize(vcf_path) / (1024*1024):.2f} MB")

# # Try loading with force_bgz and debug flags
# log_step("Loading VCF file with debug options...")
# try:
#     mt = hl.import_vcf(
#         vcf_path,
#         force_bgz=True,
#         reference_genome='GRCh38',
#         skip_invalid_loci=True,
#         min_partitions=8
#     )
    
#     # Check if loading worked
#     print("\nMatrix table schema:")
#     mt.describe()
    
#     # Try counting variants
#     n_variants = mt.count()
#     print(f"\nLoaded variants: {n_variants[0]:,}")
#     print(f"Number of samples: {n_variants[1]:,}")
    
# except Exception as e:
#     print(f"Error loading VCF: {e}")
    
#     # If that fails, try with different options
#     print("\nTrying alternative loading options...")
#     try:
#         mt = hl.import_vcf(
#             vcf_path,
#             force_bgz=True,
#             reference_genome='GRCh38',
#             skip_invalid_loci=True,
#             header_file=vcf_path  # Explicitly specify header file
#         )
#         print("Alternative loading succeeded")
#         n_variants = mt.count()
#         print(f"Loaded variants: {n_variants[0]:,}")
#     except Exception as e2:
#         print(f"Alternative loading also failed: {e2}")

Checking if file exists: True
Checking file details:
File exists: True
File size: 3376.23 MB

[13:03:00] Loading VCF file with debug options...

Matrix table schema:
----------------------------------------
Global fields:
    None
----------------------------------------
Column fields:
    's': str
----------------------------------------
Row fields:
    'locus': locus<GRCh38>
    'alleles': array<str>
    'rsid': str
    'qual': float64
    'filters': set<str>
    'info': struct {
        AC: array<int32>, 
        AN: int32, 
        AF: array<float64>, 
        grpmax: array<str>, 
        fafmax_faf95_max: array<float64>, 
        fafmax_faf95_max_gen_anc: array<str>, 
        AC_XX: array<int32>, 
        AF_XX: array<float64>, 
        AN_XX: int32, 
        nhomalt_XX: array<int32>, 
        AC_XY: array<int32>, 
        AF_XY: array<float64>, 
        AN_XY: int32, 
        nhomalt_XY: array<int32>, 
        nhomalt: array<int32>, 
        AC_afr_XX: array<int32>, 
        AF_a

+----------------+------------+----------------+---------+-------------------+
| locus          | alleles    | rsid           |    qual | filters           |
+----------------+------------+----------------+---------+-------------------+
| locus<GRCh38>  | array<str> | str            | float64 | set<str>          |
+----------------+------------+----------------+---------+-------------------+
| chr13:18173860 | ["A","C"]  | NA             |      NA | {"AS_VQSR"}       |
| chr13:18173860 | ["A","T"]  | NA             |      NA | {"AC0","AS_VQSR"} |
| chr13:18173861 | ["A","C"]  | "rs1872524921" |      NA | {"AC0","AS_VQSR"} |
| chr13:18173861 | ["A","G"]  | NA             |      NA | {"AS_VQSR"}       |
| chr13:18173861 | ["A","T"]  | NA             |      NA | {"AC0","AS_VQSR"} |
+----------------+------------+----------------+---------+-------------------+

+--------------+---------+----------------+-------------+
| info.AC      | info.AN | info.AF        | info.grpmax |
+--------------+---------+----------------+-------------+
| array<int32> |   int32 | array<float64> | array<str>  |
+--------------+---------+----------------+-------------+
| [1]          |  875186 | [1.14e-06]     | ["nfe"]     |
| [0]          |  875186 | [0.00e+00]     | NA          |
| [0]          |  885180 | [0.00e+00]     | NA          |
| [2]          |  885182 | [2.26e-06]     | ["nfe"]     |
| [0]          |  885180 | [0.00e+00]     | NA          |
+--------------+---------+----------------+-------------+

+-----------------------+-------------------------------+--------------+
| info.fafmax_faf95_max | info.fafmax_faf95_max_gen_anc | info.AC_XX   |
+-----------------------+-------------------------------+--------------+
| array<float64>        | array<str>                    | array<int32> |
+-----------------------+-------------------------------+--------------+
| NA                    | NA                            | [1]          |
| NA                    | NA                            | [0]          |
| NA                    | NA                            | [0]          |
| [5.20e-07]            | ["nfe"]                       | [0]          |
| NA                    | NA                            | [0]          |
+-----------------------+-------------------------------+--------------+

+----------------+------------+-----------------+--------------+
| info.AF_XX     | info.AN_XX | info.nhomalt_XX | info.AC_XY   |
+----------------+------------+-----------------+--------------+
| array<float64> |      int32 | array<int32>    | array<int32> |
+----------------+------------+-----------------+--------------+
| [2.34e-06]     |     426726 | [0]             | [0]          |
| [0.00e+00]     |     426726 | [0]             | [0]          |
| [0.00e+00]     |     432102 | [0]             | [0]          |
| [0.00e+00]     |     432104 | [0]             | [2]          |
| [0.00e+00]     |     432102 | [0]             | [0]          |
+----------------+------------+-----------------+--------------+

+----------------+------------+-----------------+--------------+
| info.AF_XY     | info.AN_XY | info.nhomalt_XY | info.nhomalt |
+----------------+------------+-----------------+--------------+
| array<float64> |      int32 | array<int32>    | array<int32> |
+----------------+------------+-----------------+--------------+
| [0.00e+00]     |     448460 | [0]             | [0]          |
| [0.00e+00]     |     448460 | [0]             | [0]          |
| [0.00e+00]     |     453078 | [0]             | [0]          |
| [4.41e-06]     |     453078 | [0]             | [0]          |
| [0.00e+00]     |     453078 | [0]             | [0]          |
+----------------+------------+-----------------+--------------+

+----------------+----------------+----------------+---------------------+
| info.AC_afr_XX | info.AF_afr_XX | info.AN_afr_XX | info.nhomalt_afr_XX |
+----------------+----------------+----------------+---------------------+
| array<int32>   | array<float


## 3. Quality Control
Perform basic QC steps:
- Filter to biallelic sites
- Remove missing genotypes
- Track the number of variants at each step

In [18]:
# log_step("Performing quality control...")

# # Store original counts
# original_rows = mt.count_rows()

# # Filter to biallelic sites
# mt = mt.filter_rows(hl.len(mt.alleles) == 2)
# biallelic_rows = mt.count_rows()

# # Skip the genotype filter since this is a sites-only VCF
# # Instead, let's add some other relevant filters for sites VCFs

# # Filter by quality score (if available)
# mt = mt.filter_rows(mt.qual >= 20.0)
# quality_filtered_rows = mt.count_rows()

# # Display filtering results
# print(f"Filtering results:")
# print(f"- Original variants: {original_rows:,}")
# print(f"- After biallelic filter: {biallelic_rows:,}")
# print(f"- After quality filter: {quality_filtered_rows:,}")
# print(f"- Total variants removed: {original_rows - quality_filtered_rows:,}")


[13:06:18] Performing quality control...
Filtering results:
- Original variants: 3,549,140
- After biallelic filter: 3,549,140
- After quality filter: 0
- Total variants removed: 3,549,140


In [21]:
# log_step("Extracting features from loaded VCF...")

# # First get row data and explicitly preserve keys
# try:
#     features_table = mt.rows().select(
#         # Keep keys
#         **{k: mt.rows()[k] for k in mt.rows().key},
#         # Add other fields
#         rsid = mt.rows().rsid,
#         qual = mt.rows().qual,
#         filters = mt.rows().filters,
#         # Add specific info fields
#         AC = mt.rows().info.AC[0],
#         AN = mt.rows().info.AN,
#         AF = mt.rows().info.AF[0]
#     )

#     print("\nInitial feature extraction successful")
#     count = features_table.count()
#     print(f"Number of variants: {count:,}")

#     # Calculate some basic statistics
#     stats = features_table.aggregate({
#         'mean_qual': hl.agg.stats(features_table.qual),
#         'mean_AF': hl.agg.stats(features_table.AF)
#     })

#     print("\nBasic Statistics:")
#     print(f"Total variants: {count:,}")
    
#     print(f"\nQuality Score Statistics:")
#     print(f"  Mean: {stats['mean_qual'].mean:.2f}")
#     print(f"  Std Dev: {stats['mean_qual'].stdev:.2f}")
#     print(f"  Min: {stats['mean_qual'].min:.2f}")
#     print(f"  Max: {stats['mean_qual'].max:.2f}")

#     print(f"\nAllele Frequency Statistics:")
#     print(f"  Mean: {stats['mean_AF'].mean:.6f}")
#     print(f"  Std Dev: {stats['mean_AF'].stdev:.6f}")
#     print(f"  Min: {stats['mean_AF'].min:.6f}")
#     print(f"  Max: {stats['mean_AF'].max:.6f}")

#     # Show the schema of our final table
#     print("\nFinal table structure:")
#     features_table.describe()

# except Exception as e:
#     print(f"Error in processing: {e}")
#     print("\nDebug info:")
#     print("Keys:", mt.rows().key)
#     print("\nAvailable fields:")
#     print(mt.rows().describe())


[13:09:23] Extracting features from loaded VCF...
Error in processing: 'Table.select': cannot overwrite key field 'locus' with annotate, select or drop; use key_by to modify keys.

Debug info:
Keys: <StructExpression of type struct{locus: locus<GRCh38>, alleles: array<str>}>

Available fields:
----------------------------------------
Global fields:
    None
----------------------------------------
Row fields:
    'locus': locus<GRCh38> 
    'alleles': array<str> 
    'rsid': str 
    'qual': float64 
    'filters': set<str> 
    'info': struct {
        AC: array<int32>, 
        AN: int32, 
        AF: array<float64>, 
        grpmax: array<str>, 
        fafmax_faf95_max: array<float64>, 
        fafmax_faf95_max_gen_anc: array<str>, 
        AC_XX: array<int32>, 
        AF_XX: array<float64>, 
        AN_XX: int32, 
        nhomalt_XX: array<int32>, 
        AC_XY: array<int32>, 
        AF_XY: array<float64>, 
        AN_XY: int32, 
        nhomalt_XY: array<int32>, 
        nhom